In [32]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time
from bs4 import BeautifulSoup
import pandas as pd
import re


# GENEROS Y CANCIONES

In [33]:
ranchera_links = [
    "https://www.letras.com/vicente-fernandez/el-rey/",
    "https://www.letras.com/vicente-fernandez/copa-tras-copa/",
    "https://www.letras.com/vicente-fernandez/volver-volver/",
    "https://www.letras.com/jose-alfredo-jimenez/si-nos-dejan/",
    "https://www.letras.com/vicente-fernandez/la-diferencia/",
    "https://www.letras.com/jose-alfredo-jimenez/camino-de-guanajuato/",
    "https://www.letras.com/vicente-fernandez/estos-celos/",
    "https://www.letras.com/jose-alfredo-jimenez/un-mundo-raro/",
    "https://www.letras.com/vicente-fernandez/hermoso-carino/",
    "https://www.letras.com/rocio-durcal/amor-eterno/"
]



# SCRAPEO

In [34]:

driver = webdriver.Chrome()
url = f'{ranchera_links[1]}'
driver.get(url)
# Espera a que la página cargue completamente
time.sleep(5)  # Puedes ajustar el tiempo de espera según sea necesario
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# AUTORES Y CANCIONES

In [35]:
# Obtener el nombre de la canción
song_tag = soup.find('h1', class_='textStyle-primary')
song_name = song_tag.text.strip() if song_tag else 'No encontrado'

# Obtener el nombre del autor
author_tag = soup.find('h2', class_='textStyle-secondary')
author_name = author_tag.text.strip() if author_tag else 'No encontrado'

print(f'Canción: {song_name}')
print(f'Autor: {author_name}')

Canción: Tómate una Copa
Autor: Vicente Fernández


# ESTROFAS

In [36]:
# Buscar el contenedor principal de la letra
lyric_container = soup.find('div', class_='lyric-original')

# Extraer todas las estrofas dentro de etiquetas <p>
estrofas_unicas = []
if lyric_container:
    p_tags = lyric_container.find_all('p')
    for p in p_tags:
        # Obtener el texto de la estrofa con saltos de línea reales
        texto = p.get_text(separator="\n").strip()
        # Solo agregar si no está ya en la lista
        if texto not in estrofas_unicas:
            estrofas_unicas.append(texto)

# Crear variables dinámicas estrofa1, estrofa2, etc.
for i, estrofa in enumerate(estrofas_unicas, 1):
    globals()[f'estrofa{i}'] = estrofa
    print(f'estrofa{i}:')
    print(estrofa)
    print('--------------------')

# Crear el vector final con todas las estrofas únicas
estrofas = [globals()[f'estrofa{i}'] for i in range(1, len(estrofas_unicas) + 1)]

# Si quieres verlo:
print('Vector final de estrofas únicas:')
print(estrofas)


estrofa1:
Siéntate a mi lado
Mi reciente amiga
Tomate una copa
Mientras escuchamos
Aquella canción
--------------------
estrofa2:
Tú no me conoces
Ni yo te conozco
Pero este momento
Quiero ser tu amigo
Por una ocasión
--------------------
estrofa3:
Si ves en mis ojos
Lágrimas que corren
No es que esté llorando
Es que estoy fumando
Y el humo me entró
--------------------
estrofa4:
Siéntate a mi lado
Tomate una copa
Mientras va secando
El llanto que el humo
En mis ojos dejó
--------------------
Vector final de estrofas únicas:
['Siéntate a mi lado\nMi reciente amiga\nTomate una copa\nMientras escuchamos\nAquella canción', 'Tú no me conoces\nNi yo te conozco\nPero este momento\nQuiero ser tu amigo\nPor una ocasión', 'Si ves en mis ojos\nLágrimas que corren\nNo es que esté llorando\nEs que estoy fumando\nY el humo me entró', 'Siéntate a mi lado\nTomate una copa\nMientras va secando\nEl llanto que el humo\nEn mis ojos dejó']


# DATABASE

In [37]:
# Crear un diccionario base
data = {
    'song_name': [song_name],
    'author_name': [author_name]
}

# Agregar dinámicamente las estrofas como columnas: estrofa1, estrofa2, ...
for i, estrofa in enumerate(estrofas, 1):
    columna = f'estrofa{i}'
    data[columna] = [estrofa]

# Crear el DataFrame
df = pd.DataFrame(data)

#

# FUNCTION SONGS DF

In [38]:
def scrape_lyrics_to_dataframe(links):

    driver = webdriver.Chrome()
    all_data = []

    for url in links:
        try:
            driver.get(url)
            time.sleep(3)  # Espera por si la página tarda en cargar

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            # Obtener nombre de la canción
            song_tag = soup.find('h1', class_='textStyle-primary')
            song_name = song_tag.text.strip() if song_tag else 'No encontrado'

            # Obtener nombre del autor
            author_tag = soup.find('h2', class_='textStyle-secondary')
            author_name = author_tag.text.strip() if author_tag else 'No encontrado'

            # Obtener las estrofas únicas
            lyric_container = soup.find('div', class_='lyric-original')
            estrofas_unicas = []
            if lyric_container:
                p_tags = lyric_container.find_all('p')
                for p in p_tags:
                    texto = p.get_text(separator="\n").strip()
                    if texto not in estrofas_unicas:
                        estrofas_unicas.append(texto)

            # Crear diccionario de la fila
            song_data = {
                'song_name': song_name,
                'author_name': author_name
            }

            for i, estrofa in enumerate(estrofas_unicas, 1):
                song_data[f'estrofa{i}'] = estrofa

            all_data.append(song_data)

            print(f'Scrapeado: {song_name} - {author_name}')

        except Exception as e:
            print(f'Error con URL: {url} | Error: {e}')

    driver.quit()

    # Crear DataFrame final
    df = pd.DataFrame(all_data)
    return df


In [39]:
df_letras = scrape_lyrics_to_dataframe(ranchera_links)


Scrapeado: El Rey - Vicente Fernández
Scrapeado: Tómate una Copa - Vicente Fernández
Scrapeado: Volver, Volver - Vicente Fernández
Scrapeado: No encontrado - No encontrado
Scrapeado: La Diferencia - Vicente Fernández
Scrapeado: No encontrado - No encontrado
Scrapeado: Estos Celos - Vicente Fernández
Scrapeado: Un Mundo Raro - José Alfredo Jiménez
Scrapeado: Hermoso Cariño - Vicente Fernández
Scrapeado: Amor Eterno - Rocío Durcal


In [40]:
df_letras.to_excel('letras_rancheras.xlsx', index=False)
